In [183]:
from mochi_perf.statistics import MochiStatistics

In [184]:
stats = MochiStatistics()

In [ ]:
stats.add_file("./data/test.ubuntu-vm.3384.stats.json")
# stats.add_file("./data/test.ubuntu-vm.3385.stats.json")
# stats.add_file("./data/test.ubuntu-vm.3386.stats.json")
# stats.add_file("./data/test.ubuntu-vm.3387.stats.json")
stats.origin_rpc_df.index.names


FrozenList(['file', 'address', 'name', 'rpc_id', 'provider_id', 'parent_rpc_id', 'parent_provider_id', 'sent_to'])

In [208]:
# Total number of RPCs executed
number_of_rpc_sends = total_number_of_rpcs = stats.origin_rpc_df['iforward']['duration']['num'].sum()
number_of_rpc_recvs = total_number_of_rpcs = stats.target_rpc_df['handler']['duration']['num'].sum()

assert(number_of_rpc_recvs == number_of_rpc_sends)
number_of_rpc_sends

np.int64(2)

In [207]:
# Total time spent by servers executing this RPC
# In other words, the summation of each RPC's ult time
stats.target_rpc_df["ult"]["duration"]["sum"].sum()

np.float64(0.0002141)

In [188]:
# Total time spent by clients calling this RPC
# In other words, from iforward_start to iforward_wait_end
stats.origin_rpc_df['iforward']['duration']['sum'] + stats.origin_rpc_df['iforward_wait']['relative_timestamp_from_iforward_end']['sum'] + stats.origin_rpc_df['iforward_wait']['duration']['sum']

file                            address         name  rpc_id      provider_id  parent_rpc_id  parent_provider_id  sent_to       
test.ubuntu-vm.3384.stats.json  na+sm://3384-0  echo  2924675071  65535        65535          65535               na+sm://3384-0    0.000225
                                                                               2924675071     65535               na+sm://3384-0    0.000109
Name: sum, dtype: float64

In [200]:
# Total bulk transfer time for this RPC
# i.e from itransfer to itransfer_wait_end
stats.bulk_transfer_df['itransfer']['duration']['sum'] + stats.bulk_transfer_df['itransfer_wait']['relative_timestamp_from_itransfer_end']['sum'] + stats.bulk_transfer_df['itransfer_wait']['duration']['sum']

file                            address         name  rpc_id      provider_id  parent_rpc_id  parent_provider_id  received_from   transfer_type  remote_address
test.ubuntu-vm.3384.stats.json  na+sm://3384-0  echo  2924675071  65535        65535          65535               na+sm://3384-0  pull           na+sm://3384-0    0.000004
                                                                               2924675071     65535               na+sm://3384-0  pull           na+sm://3384-0    0.000012
Name: sum, dtype: float64

In [210]:
# Total amount of data transferred using RDMA from this RPC
stats.bulk_transfer_df['itransfer']['size']['sum']

duration  \
                                                                                                                                                            num   
file                           address        name rpc_id     provider_id parent_rpc_id parent_provider_id received_from  transfer_type remote_address            
test.ubuntu-vm.3384.stats.json na+sm://3384-0 echo 2924675071 65535       65535         65535              na+sm://3384-0 pull          na+sm://3384-0        1   
                                                                          2924675071    65535              na+sm://3384-0 pull          na+sm://3384-0        1   

                                                                                                                                                                  \
                                                                                                                                                             min   
file                           address        name rpc_id     provider_id parent_rpc_id parent_provider_id received_from  transfer_type remote_address             
test.ubuntu-vm.3384.stats.json na+sm://3384-0 echo 2924675071 65535       65535         65535              na+sm://3384-0 pull          na+sm://3384-0  0.000004   
                                                                          2924675071    65535              na+sm://3384-0 pull          na+sm://3384-0  0.000012   

                                                                                                                                                                  \
                                                                                                                                                             max   
file                           address        name rpc_id     provider_id parent_rpc_id parent_provider_id received_from  transfer_type remote_address             
test.ubuntu-vm.3384.stats.json na+sm://3384-0 echo 2924675071 65535       65535         65535              na+sm://3384-0 pull          na+sm://3384-0  0.000004   
                                                                          2924675071    65535              na+sm://3384-0 pull          na+sm://3384-0  0.000012   

                                                                                                                                                                  \
                                                                                                                                                             avg   
file                           address        name rpc_id     provider_id parent_rpc_id parent_provider_id received_from  transfer_type remote_address             
test.ubuntu-vm.3384.stats.json na+sm://3384-0 echo 2924675071 65535       65535         65535              na+sm://3384-0 pull          na+sm://3384-0  0.000004   
                                                                          2924675071    65535              na+sm://3384-0 pull          na+sm://3384-0  0.000012   

                                                                                                                                                             \
                                                                                                                                                        var   
file                           address        name rpc_id     provider_id parent_rpc_id parent_provider_id received_from  transfer_type remote_address        
test.ubuntu-vm.3384.stats.json na+sm://3384-0 echo 2924675071 65535       65535         65535              na+sm://3384-0 pull          na+sm://3384-0  0.0   
                                                                          2924675071    65535              na+sm://3384-0 pull          na+sm://3384-0  0.0   

                                                                            